# DAT405 Introduction to Data Science and AI 
## 2021-2022, Reading Period 1
## Assignment 4: Spam classification using Naïve Bayes 
There will be an overall grade for this assignment. To get a pass grade (grade 5), you need to pass items 1-3 below. To receive higher grades, finish items 4 and 5 as well. 

The exercise takes place in a notebook environment where you can chose to use Jupyter or Google Colabs. We recommend you use Google Colabs as it will facilitate remote group-work and makes the assignment less technical. 
Hints:
You can execute certain linux shell commands by prefixing the command with `!`. You can insert Markdown cells and code cells. The first you can use for documenting and explaining your results the second you can use writing code snippets that execute the tasks required.  

In this assignment you will implement a Naïve Bayes classifier in Python that will classify emails into spam and non-spam (“ham”) classes.  Your program should be able to train on a given set of spam and “ham” datasets. 
You will work with the datasets available at https://spamassassin.apache.org/old/publiccorpus/. There are three types of files in this location: 
-	easy-ham: non-spam messages typically quite easy to differentiate from spam messages. 
-	hard-ham: non-spam messages more difficult to differentiate 
-	spam: spam messages 

**Execute the cell below to download and extract the data into the environment of the notebook -- it will take a few seconds.** If you chose to use Jupyter notebooks you will have to run the commands in the cell below on your local computer, with Windows you can use 7zip (https://www.7-zip.org/download.html) to decompress the data.



In [ ]:
%%script false --no-raise-error

#Download and extract data
!mkdir datasets
%cd datasets
!wget https://spamassassin.apache.org/old/publiccorpus/20021010_easy_ham.tar.bz2
!wget https://spamassassin.apache.org/old/publiccorpus/20021010_hard_ham.tar.bz2 
!wget https://spamassassin.apache.org/old/publiccorpus/20021010_spam.tar.bz2 
!tar -xjf 20021010_easy_ham.tar.bz2 
!tar -xjf 20021010_hard_ham.tar.bz2
!tar -xjf 20021010_spam.tar.bz2
%cd ..

*The* data is now in the three folders `easy_ham`, `hard_ham`, and `spam`.

In [ ]:
!ls -lah

### 1. Preprocessing: 
1.	Note that the email files contain a lot of extra information, besides the actual message. Ignore that for now and run on the entire text. Further down (in the higher-grade part), you will be asked to filter out the headers and footers. 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import os

In [ ]:
def extract_emails(directories, class_name):
  rows = []
  for directory in directories:
    for file in os.listdir(directory):
      with open(os.path.join(directory,file), encoding='latin-1') as f:
        if f is not None:
          rows.append({'email': file, 'content': f.read(), 'class': class_name})
  return pd.DataFrame(rows)

# extract emails and add class information
df_easy_ham = extract_emails(['./datasets/easy_ham'], 'ham')
df_hard_ham = extract_emails(['./datasets/hard_ham'], 'ham')
df_spam = extract_emails(['./datasets/spam'], 'spam')

# join the two dataframes
df_combined = pd.concat([df_easy_ham, df_hard_ham, df_spam])

df_combined.sample(5)

2.	We don’t want to train and test on the same data. Split the spam and the ham datasets in a training set and a test set. (`hamtrain`, `spamtrain`, `hamtest`, and `spamtest`)

In [ ]:
# pre-processing code here
X = df_combined['content']
y = df_combined['class']

# split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=0)

# print shapes
print('df_combined:', df_combined.shape)
print('X_train:', X_train.shape)
print('X_test:', X_test.shape)
print('y_train:', y_train.shape)
print('y_test:', y_test.shape)

### 2. Write a Python program that: 
1.	Uses four datasets (`hamtrain`, `spamtrain`, `hamtest`, and `spamtest`) 
2.	Trains a Naïve Bayes classifier (e.g. Sklearn) on `hamtrain` and `spamtrain`, that classifies the test sets and reports True Positive and False Negative rates on the `hamtest` and `spamtest` datasets. You can use `CountVectorizer` to transform the email texts into vectors. Please note that there are different types of Naïve Bayes Classifier in SKlearn ([Documentation here](https://scikit-learn.org/stable/modules/naive_bayes.html)). Test two of these classifiers that are well suited for this problem
- Multinomial Naive Bayes  
- Bernoulli Naive Bayes. 

Please inspect the documentation to ensure input to the classifiers is appropriate. Discuss the differences between these two classifiers. 





**Multinomial Naive Bayes**

In [ ]:
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.metrics import plot_confusion_matrix, confusion_matrix

def compare_naive_bayes(X_train, X_test, y_train, y_test, cv=None):
  # instantiate a Count Vectorizer and fit for training data
  if cv is None:
    cv = CountVectorizer()

  X_train_vector = cv.fit_transform(X_train)
  X_test_vector = cv.transform(X_test)

  # fit models using training vector
  multinomial_naive_bayes = MultinomialNB().fit(X_train_vector, y_train)
  bernoulli_naive_bayes = BernoulliNB().fit(X_train_vector, y_train)

  # make predictions on the test data
  y_pred_multinomial = multinomial_naive_bayes.predict(X_test_vector)
  y_pred_bernoulli = bernoulli_naive_bayes.predict(X_test_vector)

  # plot confusion matrices
  fig, (ax1, ax2, ax3) = plt.subplots(nrows=1, ncols=3, figsize=(20,10))
  plot_confusion_matrix(multinomial_naive_bayes, X_test_vector, y_test, ax=ax1, colorbar=None)
  ax1.set_title("Confusion Matrix Multinomial Naive Bayes")
  plot_confusion_matrix(bernoulli_naive_bayes, X_test_vector, y_test, ax=ax2, colorbar=True)
  ax2.set_title('Confusion Matrix Bernoulli Naive Bayes')
  # cf = confusion_matrix(y_test, bernoulli_naive_bayes.predict(X_test_vector))
  # plt.colorbar(cf, ax=ax3)

compare_naive_bayes(X_train, X_test, y_train, y_test), 

As the confusion matrices show, the **Multinomial Naive Bayes** model outperforms the **Bernoulli Naive Bayes** model as it suffers from way less misclassifications, and consequently classifies new emails with a higher accuracy. Interestingly enough, both models seem to correctly classify "ham" or non-spam mail with similar accuracy. It's the false negative case, when the model classifies a spam mail as "ham", that separates the two models, with the **Multinomial Naive Bayes** model being more accurate here.

### 3.Run your program on 
-	Spam versus easy-ham 


In [ ]:
def spam_vs_easy_ham(cv=None):
  spam_vs_easy_ham = pd.concat([df_spam, df_easy_ham])
  X = spam_vs_easy_ham['content']
  y = spam_vs_easy_ham['class']
  X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=0)
  compare_naive_bayes(X_train, X_test, y_train, y_test, cv=cv)

spam_vs_easy_ham()

-	Spam versus hard-ham.

In [ ]:
def spam_vs_hard_ham(cv=None):
  spam_vs_hard_ham = pd.concat([df_spam, df_hard_ham])
  X = spam_vs_hard_ham['content']
  y = spam_vs_hard_ham['class']
  X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=0)
  compare_naive_bayes(X_train, X_test, y_train, y_test, cv=cv)

spam_vs_hard_ham()

### 4.	To avoid classification based on common and uninformative words it is common to filter these out. 

**a.** Argue why this may be useful. Try finding the words that are too common/uncommon in the dataset. 

Discarding common and uninformative words removes what could be considered as noise in the data, which might help improve the accuracy of the two classification models. For instance, looking at the 20 most common words below, one notice that most of these, like `com`, `the`, `to` and `from`are either very uninformative or used in the email headers and thus likely to be irrelevant for deciding if it's spam or ham.


In [ ]:
def get_top_n_words(corpus, n=None):
    cv = CountVectorizer().fit(corpus)
    word_vector = cv.transform(corpus)
    word_sums = word_vector.sum(axis=0) 
    words_freq = [(word, word_sums[0, idx]) for word, idx in cv.vocabulary_.items()]
    words_freq = sorted(words_freq, key = lambda word: word[1], reverse=True)
    return words_freq[:n]

top_common_words = get_top_n_words(df_combined['content'])
top_common_words

**b.** Use the parameters in Sklearn’s `CountVectorizer` to filter out these words. Update the program from point 3 and run it on your data and report your results.

You have two options to do this in Sklearn: either using the words found in part (a) or letting Sklearn do it for you. Argue for your decision-making.

**Using a custom word filter**

Here one needs to specify the number of words to filter out, in order of word frequency.

In [ ]:
custom_stop_words = [word[0] for word in top_common_words[:20]]
print(f'filtered out words: {custom_stop_words}')

# generate word count vectorizer with custom in filter
cv_custom_filter = CountVectorizer(stop_words=custom_stop_words)

spam_vs_easy_ham(cv=cv_custom_filter)

Comparing it with `spam vs hard ham` dataset

In [ ]:
spam_vs_hard_ham(cv=cv_custom_filter)

**Using SKlearn's built in filtering**


In [ ]:
# generate word count vectorizer configured with built in filter
cv_built_in_filter = CountVectorizer(max_df=0.9, stop_words='english')

spam_vs_easy_ham(cv=cv_built_in_filter)

Comparing it with `spam vs hard ham` dataset

In [ ]:
spam_vs_hard_ham(cv=cv_built_in_filter)

As mentioned in the question, one can either let `sklearn` filter out common words using  the built-in stop word list for English, or one can manually specify this list. In the documentation, the authors of the `sklern` library mention there are several known issues with using the built in stop word list and that one should consider an alternative. A reason to manually specify the stop words is that this allows for experimenting with different configuration which might fit the particular dataset a lot better. On the other hand this might also increase the risk of overfitting the models with the consequence of poor accuracy when presented with new emails.

We decided to go with the built in method since it's been developed and tested over many years by more experienced engineers, as well as to avoid overfitting

### 5. Eeking out further performance
Filter out the headers and footers of the emails before you run on them. The format may vary somewhat between emails, which can make this a bit tricky, so perfect filtering is not required. Run your program again and answer the following questions: 
-	Does the result improve from 3 and 4? 
- The split of the data set into a training set and a test set can lead to very skewed results. Why is this, and do you have suggestions on remedies? 
- What do you expect would happen if your training set were mostly spam messages while your test set were mostly ham messages? 

Re-estimate your classifier using `fit_prior` parameter set to `false`, and answer the following questions:
- What does this parameter mean?
- How does this alter the predictions? Discuss why or why not.

### What to report and how to hand in.

- You will need to clearly report all results in the notebook in a clear and appropriate way, either using plots or code output (f.x. "print statements"). 
- The notebook must be reproducible, that means, we must be able to use the `Run all` function from the `Runtime` menu and reproduce all your results. **Please check this before handing in.** 
- Save the notebook and share a link to the notebook (Press share in upper left corner, and use `Get link` option. **Please make sure to allow all with the link to open and edit.**
- Edits made after submission deadline will be ignored, graders will recover the last saved version before deadline from the revisions history.
- **Please make sure all cells are executed and all the output is clearly readable/visible to anybody opening the notebook.**